# CSV to Knowledge Graph: DataFrame Transformation with Maplib

## Overview

This notebook demonstrates **DataFrame-native knowledge graph construction** using Maplib and OTTR templates. Unlike traditional RAG approaches that chunk and embed tabular data, we preserve structure by transforming DataFrames directly into RDF knowledge graphs.

## The Challenge: Tabular Data in RAG Systems

When faced with structured data (CSVs, DataFrames, databases), most RAG systems:
- 🔪 **Chunk** the data into text snippets
- 🎲 **Embed** using vector models
- 🔍 **Search** with fuzzy similarity
- 🎯 **Lose** the exact relationships and structure

But tabular data already HAS structure! Why destroy it?

## The Solution: DataFrame → Knowledge Graph

**Maplib + OTTR templates** enable:
- ✅ **Structure preservation** - Keep relationships intact
- ✅ **Type safety** - OTTR validates transformations
- ✅ **Semantic precision** - SPARQL for exact queries
- ✅ **DataFrame-native** - Stay in Polars workflow
- ✅ **Blazing fast** - Rust-backed, zero-copy operations

## The Stack Nobody's Heard Of

- **Maplib**: Rust-backed RDF construction library (from IoT world)
- **OTTR**: W3C-extension template standard for tabular → RDF transformation
- **Polars**: Fast DataFrame library with Apache Arrow backend
- **RDF/SPARQL**: W3C standards for knowledge graphs

## Breaking Best Practices

❌ Not using LangChain  
❌ Not using vector embeddings for tabular data  
❌ Not using Neo4j  
❌ Using tools from IoT/industrial domains  

✅ Preserving structure  
✅ Using formal ontologies  
✅ Querying with precision  

---

Let's transform some jaguars.


## Step 1: Install and Import Dependencies

Install Maplib and Polars if you haven't already:

```bash
pip install maplib polars
```

These are the only dependencies needed for DataFrame → Graph transformation.


In [2]:
import polars as pl
from maplib import Model
from pathlib import Path

print(f"Polars version: {pl.__version__}")


Polars version: 1.35.2


## Step 2: Load the CSV Data (Your standard data frame work flow, our happy place)

Our source data is a CSV file with jaguar conservation records, including multi-value columns (separated by `;`):

| jaguar_id | name | gender | location | monitoring_org | first_sighted | is_killed | threats | monitoring_technique | status_notes |
|-----------|------|--------|----------|----------------|---------------|-----------|---------|---------------------|--------------|
| ElJefe | El Jefe | Male | WhetstoneMountains;SantaRitaMountains | UnivOfArizona;NorthernJaguarProject | 2011-11-19 | false | | CameraTrap;ScatDetection | |
| Xama | Xamã | Male | AmazonRainforest | | | false | WildfireThreat | GPSTracking | Orphaned;Rehabilitated;Released |

This is **structured data** with 1-to-many relationships encoded as delimited strings. Perfect for knowledge graphs when properly exploded, terrible for vector embeddings.


In [16]:
# Load CSV as Polars DataFrame
csv_path = "data/jaguars.csv"
df = pl.read_csv(csv_path)

print(f"📊 Loaded {len(df)} jaguar records")
print(f"\nColumns: {', '.join(df.columns)}")

# === HANDLING LIST COLUMNS ===
# We split and explode all list columns to create a flat table.
# This generates a Cartesian product of rows, but RDF deduplication handles the redundancy.



# Define list columns to process
list_columns = ["location", "monitoring_org", "threats", "monitoring_technique"]

# Start with the base DataFrame
df_exploded = df

# Step 1: Split and explode all list columns
for col_name in list_columns:
    df_exploded = df_exploded.with_columns(
        pl.col(col_name).str.split(";")
    ).explode(col_name)





📊 Loaded 12 jaguar records

Columns: jaguar_id, name, gender, location, monitoring_org, first_sighted, is_killed, cause_of_death, identification_mark, threats, monitoring_technique, status_notes


### Why Create IRI Columns?

In a Knowledge Graph (RDF), entities are identified by **IRIs** (Internationalized Resource Identifiers), which are like unique web addresses for things.

-   **Strings ("AmazonRainforest")** are just text literals.
-   **IRIs (<http://example.org/resource#AmazonRainforest>)** are unique identifiers that can be the *subject* or *object* of a relationship.

We create these IRI columns so that:
1.  **Entities become Nodes:** "AmazonRainforest" becomes a node in the graph, not just a text label.
2.  **Connectivity:** We can attach properties (like `rdf:type` or `rdfs:label`) to these nodes.
3.  **Uniqueness:** `resource#AmazonRainforest` is the exact same node everywhere it appears, linking different jaguars to the same location.

This code block takes the clean string values and prepends a prefix (`http://example.org/resource#`) to turn them into valid IRIs for the mapping process.

In [5]:
# Step 2: Create IRI columns from the exploded string values
RES_PREFIX = "http://example.org/resource#"
df_exploded = df_exploded.with_columns([
    (pl.lit(RES_PREFIX) + pl.col("jaguar_id")).alias("id"),
    (pl.lit(RES_PREFIX) + pl.col("location").str.strip_chars()).alias("location_iri"),
    (pl.lit(RES_PREFIX) + pl.col("monitoring_org").str.strip_chars()).alias("monitoring_org_iri"),
    (pl.lit(RES_PREFIX) + pl.col("threats").str.strip_chars()).alias("threat_iri"),
    (pl.lit(RES_PREFIX) + pl.col("monitoring_technique").str.strip_chars()).alias("technique_iri")
])

print("\nExample of IRI generation:")
print(df_exploded.select([
    "location", "location_iri", 
    "monitoring_org", "monitoring_org_iri"
]).head(3))


Example of IRI generation:
shape: (3, 4)
┌────────────────────┬───────────────────────────┬──────────────────────┬──────────────────────────┐
│ location           ┆ location_iri              ┆ monitoring_org       ┆ monitoring_org_iri       │
│ ---                ┆ ---                       ┆ ---                  ┆ ---                      │
│ str                ┆ str                       ┆ str                  ┆ str                      │
╞════════════════════╪═══════════════════════════╪══════════════════════╪══════════════════════════╡
│ WhetstoneMountains ┆ http://example.org/resour ┆ UnivOfArizona        ┆ http://example.org/resou │
│                    ┆ ce#Wh…                    ┆                      ┆ rce#Un…                  │
│ WhetstoneMountains ┆ http://example.org/resour ┆ UnivOfArizona        ┆ http://example.org/resou │
│                    ┆ ce#Wh…                    ┆                      ┆ rce#Un…                  │
│ WhetstoneMountains ┆ http://example.org/resour 

### Select the final columns 


In [7]:
# Select columns for the template
# We include BOTH the IRI and the original string for each resource type
df_final = df_exploded.select([
    "id",
    "name", 
    "gender",
    "location",              # String label
    "location_iri",          # IRI
    "monitoring_org",        # String label
    "monitoring_org_iri",    # IRI
    "first_sighted",
    "is_killed",
    "cause_of_death",
    "identification_mark",
    "threats",               # String label
    "threat_iri",            # IRI
    "monitoring_technique",  # String label
    "technique_iri",         # IRI
    "status_notes"
])

print(f"\n📊 Exploded DataFrame shape: {df_final.shape}")
print(f"   (Rows increased due to Cartesian product of lists - this is expected!)")
print(f"\nFirst 3 rows (Ready for mapping):")
print(df_final.head(3))


📊 Exploded DataFrame shape: (33, 16)
   (Rows increased due to Cartesian product of lists - this is expected!)

First 3 rows (Ready for mapping):
shape: (3, 16)
┌────────────┬─────────┬────────┬────────────┬───┬────────────┬────────────┬───────────┬───────────┐
│ id         ┆ name    ┆ gender ┆ location   ┆ … ┆ threat_iri ┆ monitoring ┆ technique ┆ status_no │
│ ---        ┆ ---     ┆ ---    ┆ ---        ┆   ┆ ---        ┆ _technique ┆ _iri      ┆ tes       │
│ str        ┆ str     ┆ str    ┆ str        ┆   ┆ str        ┆ ---        ┆ ---       ┆ ---       │
│            ┆         ┆        ┆            ┆   ┆            ┆ str        ┆ str       ┆ str       │
╞════════════╪═════════╪════════╪════════════╪═══╪════════════╪════════════╪═══════════╪═══════════╡
│ http://exa ┆ El Jefe ┆ Male   ┆ WhetstoneM ┆ … ┆ null       ┆ CameraTrap ┆ http://ex ┆ null      │
│ mple.org/r ┆         ┆        ┆ ountains   ┆   ┆            ┆            ┆ ample.org ┆           │
│ esource#El ┆         ┆      

## Step 3: Initialize Maplib Model and Load Ontology

The Maplib `Model` is your in-memory knowledge graph.

Unlike GraphDB or Neo4j, there's:
- ❌ No server to start
- ❌ No HTTP requests
- ❌ No Docker containers
- ❌ No database configuration

Just pure Python, powered by Rust.

We load the **ontology** first - it defines the classes and properties (the schema).


In [9]:
# Initialize Maplib model
model = Model()

print("\n✅ Model ready for transformation!")



✅ Model ready for transformation!


In [10]:
# Auto-generate template from the EXPLODED DataFrame
template_auto = model.map_default(df_final, "id")

print("📋 Auto-generated OTTR template (from CSV):")
print(template_auto)
print("\n✅ DataFrame mapped to RDF automatically!")


📋 Auto-generated OTTR template (from CSV):
<urn:maplib_default:default_template_0> [
     <http://ns.ottr.xyz/0.4/IRI> ?id, 
     <http://www.w3.org/2001/XMLSchema#string> ?name, 
    ? <http://www.w3.org/2001/XMLSchema#string> ?gender, 
     <http://www.w3.org/2001/XMLSchema#string> ?location, 
     <http://ns.ottr.xyz/0.4/IRI> ?location_iri, 
    ? <http://www.w3.org/2001/XMLSchema#string> ?monitoring_org, 
    ? <http://ns.ottr.xyz/0.4/IRI> ?monitoring_org_iri, 
    ? <http://www.w3.org/2001/XMLSchema#string> ?first_sighted, 
     <http://www.w3.org/2001/XMLSchema#boolean> ?is_killed, 
    ? <http://www.w3.org/2001/XMLSchema#string> ?cause_of_death, 
    ? <http://www.w3.org/2001/XMLSchema#string> ?identification_mark, 
    ? <http://www.w3.org/2001/XMLSchema#string> ?threats, 
    ? <http://ns.ottr.xyz/0.4/IRI> ?threat_iri, 
    ? <http://www.w3.org/2001/XMLSchema#string> ?monitoring_technique, 
    ? <http://ns.ottr.xyz/0.4/IRI> ?technique_iri, 
    ? <http://www.w3.org/2001/XMLSc

### What Just Happened?

**Look at the template above!** Maplib:
- Detected IRIs vs literals vs booleans
- Created parameters with proper XSD types
- Generated `ottr:Triple()` statements for each column
- Marked optional parameters (like `cause_of_death`) with `?`

**The catch:** Predicates are generic (`urn:maplib_default:gender` instead of `ont:hasGender`)

Let's query it first to see what we get!


In [17]:
# Query the auto-generated graph
query_generic = """
SELECT ?id ?name ?location WHERE {
    ?id <urn:maplib_default:name> ?name .
    ?id <urn:maplib_default:location_iri> ?location .
}
LIMIT 5
"""

result = model.query(query_generic)
print("🔍 Querying with generic predicates:")
result


🔍 Querying with generic predicates:


id,name,location
str,str,str
"""<http://example.org/resource#A…","""Asa""","""<http://example.org/resource#P…"
"""<http://example.org/resource#C…","""Cayenita""","""<http://example.org/resource#H…"
"""<http://example.org/resource#C…","""Cochise""","""<http://example.org/resource#H…"
"""<http://example.org/resource#E…","""El Jefe""","""<http://example.org/resource#S…"
"""<http://example.org/resource#E…","""El Jefe""","""<http://example.org/resource#S…"


## Step 5: Refine Template with Semantic Predicates ✨

**The Workflow: From Auto to Semantic**

```
1️⃣ map_default(df, "id")
   ↓
   Generates template with generic predicates
   (urn:maplib_default:name, urn:maplib_default:gender, etc.)

2️⃣ Copy template structure
   ↓
   Take parameter types and structure

3️⃣ Replace predicates with semantic ones
   ↓
   urn:maplib_default:gender → ont:hasGender
   urn:maplib_default:location_iri → ont:occursIn

4️⃣ Add template to model and map
   ↓
   Same structure, semantic meaning!
```

**Key Insight**: `map_default()` gives you the **scaffolding** (types, structure).  
You provide the **semantics** (meaningful predicates from your ontology).

**Important: Defining IRI Resources**  
When working with multi-value columns that become IRIs (locations, organizations, threats, techniques), we need to define **both**:
1. The **relationship** from the jaguar to the resource (e.g., `ont:occursIn`)
2. The **resource itself** with its properties:
   - `rdf:type` - What kind of thing is it? (e.g., `ont:Location`, `ont:Threat`)
   - `rdfs:label` - Human-readable name

Why? Because `:WhetstoneMountains` is not just a pointer - it's an entity in the knowledge graph that should have its own semantic information. This allows queries like "Find all Locations" or "What is the label of this threat?" to work correctly.

Let's create a refined template based on the auto-generated structure:


In [18]:
# Create COMPREHENSIVE refined template
# This template handles ALL properties in one go.
# Maplib will apply this to every row of the exploded DataFrame.
# Redundant triples (e.g. name, gender) will be deduplicated automatically.

template_refined = """
@prefix ont: <http://example.org/ontology#> .
@prefix res: <http://example.org/resource#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .

ont:JaguarComprehensive [
  <http://ns.ottr.xyz/0.4/IRI> ?id,
  <http://www.w3.org/2001/XMLSchema#string> ?name,
  ? <http://www.w3.org/2001/XMLSchema#string> ?gender,
  ? <http://www.w3.org/2001/XMLSchema#string> ?location,
  ? <http://ns.ottr.xyz/0.4/IRI> ?location_iri,
  ? <http://www.w3.org/2001/XMLSchema#string> ?monitoring_org,
  ? <http://ns.ottr.xyz/0.4/IRI> ?monitoring_org_iri,
  ? <http://www.w3.org/2001/XMLSchema#string> ?first_sighted,
  <http://www.w3.org/2001/XMLSchema#boolean> ?is_killed,
  ? <http://www.w3.org/2001/XMLSchema#string> ?cause_of_death,
  ? <http://www.w3.org/2001/XMLSchema#string> ?identification_mark,
  ? <http://www.w3.org/2001/XMLSchema#string> ?threats,
  ? <http://ns.ottr.xyz/0.4/IRI> ?threat_iri,
  ? <http://www.w3.org/2001/XMLSchema#string> ?monitoring_technique,
  ? <http://ns.ottr.xyz/0.4/IRI> ?technique_iri,
  ? <http://www.w3.org/2001/XMLSchema#string> ?status_notes
] :: {
  ottr:Triple(?id, a, ont:Jaguar),
  ottr:Triple(?id, rdfs:label, ?name),
  ottr:Triple(?id, ont:scientificName, "Panthera onca"),
  ottr:Triple(?id, ont:hasGender, ?gender),
  ottr:Triple(?id, ont:occursIn, ?location_iri),
  ottr:Triple(?location_iri, a, ont:Location),
  ottr:Triple(?location_iri, rdfs:label, ?location),
  ottr:Triple(?id, ont:monitoredByOrg, ?monitoring_org_iri),
  ottr:Triple(?monitoring_org_iri, a, ont:ConservationOrganization),
  ottr:Triple(?monitoring_org_iri, rdfs:label, ?monitoring_org),
  ottr:Triple(?id, ont:hasMonitoringStartDate, ?first_sighted),
  ottr:Triple(?id, ont:wasKilled, ?is_killed),
  ottr:Triple(?id, ont:causeOfDeath, ?cause_of_death),
  ottr:Triple(?id, ont:hasIdentificationMark, ?identification_mark),
  ottr:Triple(?id, ont:facesThreat, ?threat_iri),
  ottr:Triple(?threat_iri, a, ont:Threat),
  ottr:Triple(?threat_iri, rdfs:label, ?threats),
  ottr:Triple(?id, ont:monitoredByTechnique, ?technique_iri),
  ottr:Triple(?technique_iri, a, ont:MonitoringTechnique),
  ottr:Triple(?technique_iri, rdfs:label, ?monitoring_technique),
  ottr:Triple(?id, rdfs:comment, ?status_notes),
  ottr:Triple(?id, ont:hasDietType, res:CarnivoreDiet)
} .
"""

print("📋 Refined OTTR template created (Comprehensive with labels + types)!")
print("   - Includes rdfs:label triples for all IRI resources")
print("   - Includes rdf:type triples for all IRI resources")


📋 Refined OTTR template created (Comprehensive with labels + types)!
   - Includes rdfs:label triples for all IRI resources
   - Includes rdf:type triples for all IRI resources


In [19]:
# Create a new model for refined mapping
model_refined = Model()

#load the ontology into the model
ontology_path="data/jaguar_ontology.ttl"
model_refined.read(ontology_path, format="turtle")
print(f"\n🧬 Loaded ontology from {ontology_path}")
# Add the comprehensive template
model_refined.add_template(template_refined)

# Map the EXPLODED data
print(f"🔄 Transforming {len(df_final)} rows with comprehensive template...")
model_refined.map("http://example.org/ontology#JaguarComprehensive", df_final)

print("✅ Data mapped!")
print("   (Duplicate triples were automatically handled by RDF set semantics)")



print("\n🎉 Your DataFrame is now a semantic knowledge graph!")



🧬 Loaded ontology from data/jaguar_ontology.ttl
🔄 Transforming 33 rows with comprehensive template...
✅ Data mapped!
   (Duplicate triples were automatically handled by RDF set semantics)

🎉 Your DataFrame is now a semantic knowledge graph!


## Step 7: Query with Semantic SPARQL 🎯

Now we can query with **ontology-aligned predicates** - this is proper semantic querying!

### Query 1: Count All Jaguars


In [20]:
query = """
PREFIX ont: <http://example.org/ontology#>
SELECT (COUNT(?jaguar) as ?count) WHERE {
    ?jaguar a ont:Jaguar .
}
"""

result = model_refined.query(query)
print("📊 Total jaguars in knowledge graph:")
result


📊 Total jaguars in knowledge graph:


count
u32
12


### Query 2: List All Jaguar Names and Genders


In [21]:
query = """
PREFIX ont: <http://example.org/ontology#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT ?name ?gender ?location WHERE {
    ?jaguar a ont:Jaguar .
    ?jaguar rdfs:label ?name .
    OPTIONAL { ?jaguar ont:hasGender ?gender . }
    OPTIONAL { ?jaguar ont:occursIn ?loc . ?loc rdfs:label ?location . }
}
ORDER BY ?name
"""

result = model_refined.query(query)
print("🐆 Jaguar roster:")
result


🐆 Jaguar roster:


name,gender,location
str,str,str
"""Asa""","""Female""","""Pantanal"""
"""Cayenita""","""Female""","""HatoLaAurora"""
"""Cochise""","""Male""","""HuachucaMountains"""
"""El Jefe""","""Male""","""SantaRitaMountains"""
"""El Jefe""","""Male""","""Sonora"""
…,…,…
"""Sombra""","""Male""","""ChiricahuaMountains"""
"""Sombra""","""Male""","""DosCabezasMountains"""
"""Xamã""","""Male""","""AmazonRainforest"""


### Query 3: Find Killed Jaguars and Causes


In [22]:
query = """
PREFIX ont: <http://example.org/ontology#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT ?name ?cause WHERE {
    ?jaguar a ont:Jaguar .
    ?jaguar ont:wasKilled true .
    OPTIONAL { ?jaguar rdfs:label ?name . }
    OPTIONAL { ?jaguar ont:causeOfDeath ?cause . }
}
"""

result = model_refined.query(query)
print("💔 Jaguars that were killed:")
result


💔 Jaguars that were killed:


name,cause
str,str
"""Macho B""","""Euthanasia following kidney fa…"
"""Yo'oko""","""Killed by mountain lion hunter"""


In [23]:
query = """
PREFIX ont: <http://example.org/ontology#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT ?name ?org ?location ?firstSighted WHERE {
    ?jaguar a ont:Jaguar .
    ?jaguar rdfs:label ?name .
    OPTIONAL {?jaguar ont:monitoredByOrg ?orgRes . 
    ?orgRes rdfs:label ?org .}
    ?jaguar ont:occursIn ?locRes .
    ?locRes rdfs:label ?location .
    OPTIONAL { ?jaguar ont:hasMonitoringStartDate ?firstSighted . }
}
ORDER BY ?org ?location
"""

result = model_refined.query(query)
print("🔗 Jaguars with organizational and location context:")
result


🔗 Jaguars with organizational and location context:


name,org,location,firstSighted
str,str,str,str
"""Xamã""",null,"""AmazonRainforest""",null
"""O:ṣhad Ñu:kudam""",null,"""Arizona""",null
"""Macho B""",null,"""BaboquivariMountains""","""1996-01-01"""
"""Sombra""",null,"""ChiricahuaMountains""","""2016-11-01"""
"""Sombra""",null,"""DosCabezasMountains""","""2016-11-01"""
…,…,…,…
"""Ferinha""","""PantanalJaguarProject""","""Pantanal""","""2012-09-10"""
"""Mariposa""","""PantheraColombia""","""HatoLaAurora""","""2009-01-01"""
"""El Jefe""","""UnivOfArizona""","""SantaRitaMountains""","""2011-11-19"""


## Step 8: Export the Knowledge Graph

You can export your knowledge graph to Turtle format for:
- Importing into GraphDB or other triple stores
- Sharing with collaborators
- Version control
- Backup


In [24]:
# Export to Turtle
output_path = "data/exported_jaguars.ttl"
model_refined.write(output_path, format="turtle")

print(f"💾 Knowledge graph exported to {output_path}")
print("\n📤 You can now import this into GraphDB or other triple stores")


💾 Knowledge graph exported to data/exported_jaguars.ttl

📤 You can now import this into GraphDB or other triple stores


---

## Alternative Approach: Load Pre-Written OTTR Template from File

The auto-generate → refine workflow above is perfect for exploration and iteration.

But if you already have a **pre-written OTTR template file** (e.g., `jaguar_template.ottr`), you can skip the generation step and load it directly.

This is useful when:
- You have established templates to reuse
- You're working in production with stable schemas
- You want version-controlled templates

### Load OTTR Template from File


In [25]:
# Create a new model for file-based template approach
model_file = Model()
model_file.read(ontology_path, format="turtle")

# Load OTTR template from file
template_path = "data/jaguar_template.ottr"
with open(template_path, 'r', encoding='utf-8') as f:
    ottr_template = f.read()

# Add template to model
model_file.add_template(ottr_template)

print(f"📋 Loaded OTTR template from {template_path}")
print(f"   Template length: {len(ottr_template)} characters")
print("\n📌 Template defines:")
print("   - ont:JaguarInstance  OTTR-template")



📋 Loaded OTTR template from data/jaguar_template.ottr
   Template length: 2263 characters

📌 Template defines:
   - ont:JaguarInstance  OTTR-template


### Map DataFrame with File Template


In [26]:
# Map DataFrame to RDF using template from file
template_iri = "http://example.org/ontology#JaguarInstance"

print(f"🔄 Transforming {len(df_final)} rows with file template...")
model_file.map(template_iri, df_final)
print("✅ Transformation complete!")

# Quick query to verify
result = model_file.query("""
PREFIX ont: <http://example.org/ontology#>
SELECT (COUNT(?j) as ?count) WHERE { ?j a ont:Jaguar . }
""")
print("\n🎉 File template approach works too!")
result


🔄 Transforming 33 rows with file template...
✅ Transformation complete!

🎉 File template approach works too!


count
u32
12


## Bonus: Load Additional Instance Data

You can also load additional RDF data that wasn't created from CSV.

This shows the power of RDF: **multiple data sources, single graph**.


In [63]:
# Load existing instance data into the refined model
instances_path = "data/jaguars.ttl"
model_refined.read(instances_path, format="turtle")

print(f"🐆 Loaded additional instances from {instances_path}")
print("\n🔗 Now your graph contains:")
print("   - CSV-derived jaguars (from Maplib transformation)")
print("   - Manually curated jaguars (from jaguars.ttl)")
print("   - All queryable together!")

# Count total jaguars now
result = model_refined.query("""
PREFIX ont: <http://example.org/ontology#>
SELECT (COUNT(?j) as ?total) WHERE { ?j a ont:Jaguar . }
""")
print("\n📊 Total jaguars after merging:")
result


🐆 Loaded additional instances from data/jaguars.ttl

🔗 Now your graph contains:
   - CSV-derived jaguars (from Maplib transformation)
   - Manually curated jaguars (from jaguars.ttl)
   - All queryable together!

📊 Total jaguars after merging:


total
u32
20


---

## Summary: Auto-Generate vs. Pre-Written Templates

| Aspect | Auto-Generate → Refine | Load from File |
|--------|----------------------|----------------|
| **Setup Time** | Fast (seconds) | Instant (if file exists) |
| **Property Names** | Start generic → refine to semantic | Semantic from start |
| **Type Safety** | Auto-inferred → manual refinement | Fully controlled |
| **Ontology Compliance** | Requires refinement step | Perfect alignment (if well-written) |
| **Use Case** | Exploration, new datasets | Production, established schemas |
| **Flexibility** | See structure, iterate quickly | Need to edit file to change |

## Recommended Workflow:

```
1️⃣ New dataset?
   → Use map_default() to explore
   → Refine template with semantic predicates
   → Save refined template to file

2️⃣ Production pipeline?
   → Load pre-written template from file
   → Map all incoming data consistently
   → Version control your templates
```

## The Power of This Approach:

✅ **Stay in DataFrames** - No need to learn new tools  
✅ **Preserve structure** - Don't destroy relationships with embeddings  
✅ **Query with precision** - SPARQL > fuzzy vector search  
✅ **Blazing fast** - Rust-backed, zero-copy operations  
✅ **Standards-based** - W3C OTTR, RDF, SPARQL  

**This is what structured Graph RAG should look like.** 🎯
